In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mwxml
from src.mongo_utils import collection, extract_content
from tqdm.notebook import tqdm

In [3]:
def extract_text(page):
    """Extracts the text content from a MediaWiki page."""
    for revision in page:
        if hasattr(revision, 'text') and revision.text is not None:
            return revision.text
        elif hasattr(revision, 'slots') and 'main' in revision.slots:
            return revision.slots['main'].text

def process_dump(xml_dump_path):
    dump = mwxml.Dump.from_file(open(xml_dump_path, 'rb'))

    for page in tqdm(dump):
        text = extract_text(page)
        if text:
            article_data = {
                'title': page.title,
                'text': text
            }
            collection.insert_one(article_data)

xml_dump_path = "dewiktionary-20240101-pages-articles.xml"

process_dump(xml_dump_path)

0it [00:00, ?it/s]

In [27]:
import re
from collections import defaultdict

def get_entries(text, token):
    if token not in text:
        return []
    search_token = '{{' + token + '}}'
    
    m = re.search(search_token+'\s*:(.*?)(?=\n{{[^{}]+}}|\Z)', text, re.DOTALL)
    if not m:
        return []
    m = m.group()
    m = m.replace(search_token+'\n', '')
    lines = list(filter(lambda item: item.strip(), m.split("\n")))
    return lines

def extract_entries(entries):
    curr_id = -1
    bedeutungen = defaultdict(lambda: [])
    for i, line in enumerate(entries):
        if re.search('{{Beispiele fehlen.*}}', line):
            continue
        if m := re.search(':\[[0-9]+\]', line):
            curr_id = int(re.search('[0-9]+', m.group()).group())
            entry = line.replace(f':[{curr_id}]', '').strip()
            if entry:
                bedeutungen[str(curr_id)] += [entry]
        elif m := re.search('::— ', line):
            if curr_id != -1:
                bedeutungen[str(curr_id)] += [line.replace('::— ', '').strip()]
    return bedeutungen

In [ ]:
from tqdm.notebook import tqdm


search_query =  {'text': {"$regex": ".*{{Beispiele}}.*", "$options": "i"}}
result = collection.find(search_query)
count = 0

empty_words = []
available_words = []

i = 0
for doc in tqdm(result):
    insert_data = {}
    a = get_entries(doc['text'], 'Bedeutungen')
    bedeutungen = extract_entries(a)
    insert_data['bedeutungen'] = bedeutungen
    a = get_entries(doc['text'], 'Beispiele')
    beispiele = extract_entries(a)
    insert_data['beispiele'] = beispiele

    if beispiele and bedeutungen:
        available_words += [doc['title']]
    else:
        empty_words += [doc['title']]
        continue
    result = collection.update_one({"_id": doc['_id']}, {'$set': insert_data})
count

0it [00:00, ?it/s]

In [5]:
from tqdm.notebook import tqdm
from src.mongo_utils import clear_database

clear_database()

search_query =  {'text': {"$regex": ".*{{Beispiele}}.*", "$options": "i"}}
result = collection.find(search_query)
count = 0

empty_words = []
available_words = []

i = 0
for doc in tqdm(result):
    insert_data = {}
    bedeutungen, beispiele = extract_content(doc['text'])
    insert_data['bedeutungen'] = bedeutungen
    insert_data['beispiele'] = beispiele

    if beispiele and bedeutungen:
        count += 1
        available_words += [doc['title']]
    else:
        empty_words += [doc['title']]
        continue
    result = collection.update_one({"_id": doc['_id']}, {'$set': insert_data})
count

0it [00:00, ?it/s]

155881

In [113]:
extract_content(doc['text'])

UnboundLocalError: local variable 'index_2' referenced before assignment

In [26]:
beispiele

defaultdict(<function __main__.extract_entries.<locals>.<lambda>()>,
            {'1': ["Die Lamprologini sind eine ''Tribus'' der Buntbarsche (Cichlidae).<ref>{{Wikipedia|Lamprologini}}</ref>"],
             '2': ["Jede ''Tribus'' untergliederte sich in fünf centuriae seniorum und fünf centuriae iuniorum.<ref>{{Wikipedia|Tribus (Rom)}}</ref>"]})

In [8]:
bedeutungen

defaultdict(<function __main__.extract_entries.<locals>.<lambda>()>,
            {'5': ['junge Kalmuspflanze']})

In [260]:
doc

{'_id': ObjectId('65a93a340d199cc9e0c9c5b7'),
 'title': 'familia',
 'text': '{{Siehe auch|[[família]]}}\n== familia ({{Sprache|Latein}}) ==\n=== {{Wortart|Substantiv|Latein}}, {{f}} ===\n\n{{Latein Substantiv Übersicht|familia|famili|ae}}\n\n{{Worttrennung}}\n:fa·mi·lia, {{Pl.}} fa·mi·li·ae\n\n{{Grammatische Merkmale}}\n:[[Genitiv]] Singular nach [[pater]], [[mater]], [[filius]], [[filia]]: \'\'familiās\'\'<ref name="Georges">{{Ref-Georges}}</ref>\n\n{{Bedeutungen}}\n:[1] das [[Haus]], der [[Hausstand]] (mit Sklaven)\n:[2] das [[Geschlecht]], die [[Familie]]\n:[3] die [[Dienerschaft]]\n\n{{Herkunft}}\n:zu dem Adjektiv \'\'{{Ü|la|famulus}}\'\'<ref name="Georges" /> „dienend“; ursprüngliche Bedeutung: die Gesamtheit der unter einem \'\'[[dominus]]\'\' stehenden Sklaven, besonders als Angehörige und Teil der Familie, dann übertragen die ganze Hausgenossenschaft (Freie und Sklaven)\n\n{{Beispiele}}\n:[1] {{Beispiele fehlen|spr=la}}\n\n==== {{Übersetzungen}} ====\n{{Ü-Tabelle|1|G=das Haus, 

In [263]:
len(available_words)

85950

In [264]:
len(empty_words)

160750

In [126]:
example_1 = str(doc['text'])

In [5]:


a = get_entries(doc['text'], 'Bedeutungen')
bedeutungen = extract_entries(a)
a = get_entries(doc['text'], 'Beispiele')
beispiele = extract_entries(a)

print(bedeutungen)
print(beispiele)

defaultdict(<function extract_entries.<locals>.<lambda> at 0x7f76d2ce6e60>, {'1': ['lautes Rufen; fröhliches, lautes Durcheinander']})
defaultdict(<function extract_entries.<locals>.<lambda> at 0x7f76d2ce72e0>, {'1': ["Als er die Treppe hinaufkam, wurde er mit großem ''Hallo'' empfangen.", "„Da gab es keine Möglichkeit, mich von vielen Leuten (…) zu verabschieden. Deswegen wird es jetzt ein Tag des großen ''Hallos'' und des offiziellen Abschieds werden.“<ref>Marc Heinrich (Interviewer): [http://www.faz.net/aktuell/rhein-main/sport/eintracht-frankfurt/im-gespraech-armin-reutershahn-der-tag-des-hallos-und-des-abschieds-1843358.html ''Im Gespräch: Armin Reutershahn. „Der Tag des Hallos und des Abschieds“''] auf www.faz.net vom 15. August 2009</ref>"]})


In [258]:
a

[':[1] {{Beispiele fehlen|spr=la}}',
 ":[6] ''Ordines'' sunt exempli gratia Insectis Diptera et Odonata.",
 "::''Ordnungen'' der Insekten sind zum Beispiel die Zweiflügler und die Libellen."]

In [225]:
import mwparserfromhell

wikicode = mwparserfromhell.parse(doc['text'])
for template in wikicode.filter_templates():
    if template.name.matches("Bedeutungen"):
        bedeutungen_template = template
        break
    else:
        print(template.name)

Shortcut
Linkbox Regeln und Konventionen/Formatierung


In [224]:
print(doc['text'])

{{Shortcut|[[WT:FV]]}}
{{Linkbox Regeln und Konventionen/Formatierung}}
<div style="background-color:#FFFFFF; border: 3px solid #dfdfdf; padding: 1em 1em 1em 1em;">
Um diese Formatvorlage zu verwenden, kann man auch einfach '''<code><nowiki>{{subst:Formatvorlage}}</nowiki></code>''' im Bearbeitungsfenster eingeben. Weitere Hinweise finden sich unter '''[[Hilfe:Verwenden der Formatvorlage|Verwenden der Formatvorlage]]'''.</div>
[[Kategorie:Wiktionary|Formatvorlage]]

<br />
* Wenn du Hilfe für die einzelnen Abschnitte brauchen solltest, siehe unter [[Hilfe:Formatvorlage]]
* Solltest du gleich einen Eintrag erstellen wollen, siehe unter [[Wiktionary:Eintrag erstellen]]
* Für andere Wortarten als Substantive (Verben, Adjektive und andere) finden sich unter [[Hilfe:Flexionstabellen|Flexionstabellen]] Tabellen zum Herauskopieren für die verbreitetsten Fälle
* Für Wörter aus anderen Sprachen gibt es bereits eine Anzahl eigener Formatvorlagen, siehe [[Wiktionary:Eintrag erstellen]]

<br style

In [100]:
for line in a:
    print(line)

:[1] ''gegenständlich:'' [[Reihe]], [[Sitzreihe]] ''(Reihe der Sitze oder Bänke im Theater)'', [[Lage]], [[Schicht]]
:[2] ''militärisch:''
::— ''(abstrakt: Militär-Gruppierung, Legion)'' [[Glied]], [[Zenturie]], [[Abteilung]], [[Abteilungsglied]], [[Kompanie]]
::— ''(personifiziert)'' der [[Hauptmann]] selbst
:[3] ''politisch-gesellschaftlich:'' [[Stand]], [[Klasse]], [[Rang]], [[Stellung]]
:[4] ''abstrakt:'' [[Ordnung]], gehörige Reihenfolge, [[Anordnung]], [[Regel]]
::— ''übertragen:'' [[Verfassung]], [[Zustand]]
:[5] ''kirchlich (mittellateinisch):''
::[5a] [[Ordensgemeinschaft]] ''von Mönchen''
::[5b] kirchlicher Stand: [[Weihestufe]]
::[5c] die (göttliche) Weltordnung
:[6] ''[[Taxonomie]]: Biologische Systematik (neulateinisch):'' [[Ordnung]] oder fachwissenschaftlich [[Ordo]] ''(Zusammenfassung mehrerer eng verwandter Familien, Teil eine Klasse)''



In [83]:
repr(re.search('[0-9]+', ':[11]').group())

"'11'"